In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Load dataset
df = pd.read_csv("/content/drive/MyDrive/fsdbus/stMTMV_water_quality_dataset.csv")

# Encode target variable: Safe → 0, Unsafe → 1
df['Water_Quality_Label'] = df['Water_Quality_Label'].map({'Safe': 0, 'Unsafe': 1})

# Selected features (adjust if needed)
selected_features = [
    "Temperature", "pH", "Chlorine_Level",
    "Pipe_Age", "Pipe_Length", "Pipe_Diameter",
    "Turbidity", "Conductivity", "Water_Pressure"
]

# Feature matrix and target
X = df[selected_features]
y = df['Water_Quality_Label']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Predict on test set
y_pred = lr_model.predict(X_test_scaled)

# Evaluation metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("🔍 Model Evaluation")
print(f"📉 Mean Squared Error: {mse:.4f}")
print(f"📏 Mean Absolute Error: {mae:.4f}")
print(f"📊 R² Score: {r2:.4f}")

# Save model and scaler
joblib.dump(lr_model, "/content/drive/MyDrive/lr/LinearRegression_WaterQualityModel.pkl")
joblib.dump(scaler, "/content/drive/MyDrive/lr/LinearRegression_Scaler.save")

print("\n✅ Model and scaler saved to /content/drive/MyDrive/lr/")


🔍 Model Evaluation
📉 Mean Squared Error: 0.2507
📏 Mean Absolute Error: 0.5003
📊 R² Score: -0.0031

✅ Model and scaler saved to /content/drive/MyDrive/lr/


In [7]:
# Imports
import numpy as np
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load trained model and scaler
lr_model = joblib.load("/content/drive/MyDrive/lr/LinearRegression_WaterQualityModel.pkl")
scaler = joblib.load("/content/drive/MyDrive/lr/LinearRegression_Scaler.save")

# Feature names (must match model training)
feature_names = [
    "Temperature", "pH", "Chlorine_Level",
    "Pipe_Age", "Pipe_Length", "Pipe_Diameter",
    "Turbidity", "Conductivity", "Water_Pressure"
]

# Create input widgets
input_widgets = {
    name: widgets.FloatText(
        description=name,
        layout=widgets.Layout(width='400px'),
        style={'description_width': '250px'}
    ) for name in feature_names
}

# Create prediction button and output box
submit_button = widgets.Button(description="Predict Water Quality", button_style='success')
output = widgets.Output()

# Define prediction logic
def on_predict_clicked(b):
    with output:
        clear_output()
        try:
            # Collect input values
            values = [input_widgets[name].value for name in feature_names]
            sample = np.array([values])
            sample_scaled = scaler.transform(sample)

            # Predict water quality score (regression output)
            prediction = lr_model.predict(sample_scaled)[0]
            prediction_rounded = round(prediction, 3)
            predicted_class = int(prediction >= 0.5)

            print("🔍 Prediction Results")
            print(f"📈 Predicted Score: {prediction_rounded}")
            print("💧 Predicted Water Quality:", "✅ Safe (0)" if predicted_class == 0 else "❌ Unsafe (1)")

        except Exception as e:
            print("❌ Error:", str(e))

# Bind and display UI
submit_button.on_click(on_predict_clicked)
display(widgets.VBox(list(input_widgets.values()) + [submit_button, output]))
